## Population growth
* Populations that die are deleted, so they don't grow.
* However all pops that can grow will, regardless of wheter they are starving or not. 
* Starving will lower thier health, which will eventually make them unable to grow. 

Sarving and Death is taken care of elsewhere, so this notebook won't have that filter. 


In [1]:
import sys
import numpy as np
import pandas as pd

# mapping to the modules that make the app
sys.path.insert(0, "..")
sys.path.insert(0, "../..")
sys.path.insert(0, "../../app")


import yaml, ssl, asyncio, pickle, os, ast

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

#importing the libraries from the app
from app.connectors import cmdb_graph
from app.objects import time
from app.objects import population
from app.functions import language
from helpers import test_queries

c = cmdb_graph.CosmosdbClient()


executing local windows deployment


Much of the Population growth is handled in scripts inside the Azure Function

In [2]:
params = yaml.safe_load(open(os.path.join(os.getenv("abspath"),"app/configurations/popgrowthconfig.yaml")))
syllables = pickle.load(open(os.path.join(os.getenv("abspath"),"app/creators/specs/syllables.p"), "rb"))
username = 'notebook'

Time is updated by the `time` function. It is always running.

In [3]:
t = time.Time(c)
t.get_current_UTU()
params['currentTime'] = t.params['currentTime']
t

< time at: 2023-09-04T20:30:58.848919+00:00 UTU:1001 >

In [4]:
params

{'pop_health_requirement': 0.6,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty'],
 'currentTime': 1001}

For this notebook, setting the `pop_health_requirement` to an arbitrary value just to examine.

In [5]:
params['pop_health_requirement'] = 0.4

In [6]:
# from tools import growth
global_health_manager = population.Global_Pop_Manager(params,c)


In [7]:
global_health_manager.get_pop_health()

pops_df = pd.DataFrame([d['pop'] for d in global_health_manager.data])
species_df = pd.DataFrame([d['species'] for d in global_health_manager.data])
locations_df = pd.DataFrame([d['location'] for d in global_health_manager.data])
factions_df = pd.DataFrame([d['faction'] for d in global_health_manager.data])

In [8]:
# pops_df,species_df,locations_df,factions_df = growth.get_pop_health(c,params)
print(f"lenth of pops == 0 : {len(pops_df)==0}")
pops_df

lenth of pops == 0 : False


,health,name,objid,conformity,literacy,aggression,constitution,isIn,industry,wealth,factionLoyalty,isIdle,username,objtype,id
0,0.7,Tadtperei Wal,9182913651185,0.386,0.484,0.533,0.194,3944934639394,0.3635,0.4238,0.382,true,BillmanLocal2,pop,9182913651185
1,0.7,Harpo Folamso,6961571307402,0.578,0.393,0.345,0.738,3078395587160,0.5415,0.4672,0.490,true,BillmanLocal2,pop,6961571307402
2,0.7,Harpo Sidinco,2033031753966,0.277,0.552,0.320,0.541,3078395587160,0.4305,0.4912,0.238,true,BillmanLocal2,pop,2033031753966
3,0.7,Harpo Hal,1310302536813,0.439,0.415,0.468,0.643,3078395587160,0.5555,0.4852,0.669,true,BillmanLocal2,pop,1310302536813
4,0.7,Harpo Manburselto,3770768094710,0.364,0.551,0.548,0.529,3078395587160,0.5385,0.5448,0.081,true,BillmanLocal2,pop,3770768094710
5,0.7,Harpo Pingfrankayyo,4343184883492,0.456,0.656,0.437,0.498,3078395587160,0.4675,0.5618,0.577,true,BillmanLocal2,pop,4343184883492
6,0.7,Tadtperei Melgretanhor,7006358225710,0.485,0.484,0.666,0.326,3944934639394,0.4960,0.4900,0.295,true,BillmanLocal2,pop,7006358225710


In [9]:
species_df

,name,objid,consumes,effuses,conformity,aggression,literacy,constitution,username,objtype,id
0,Narbiakiy,5717091433803,organics,"organic waste, plastics",0.5,0.5,0.5,0.5,BillmanLocal2,species,5717091433803
1,Narbiakiy,5717091433803,organics,"organic waste, plastics",0.5,0.5,0.5,0.5,BillmanLocal2,species,5717091433803
2,Narbiakiy,5717091433803,organics,"organic waste, plastics",0.5,0.5,0.5,0.5,BillmanLocal2,species,5717091433803
3,Narbiakiy,5717091433803,organics,"organic waste, plastics",0.5,0.5,0.5,0.5,BillmanLocal2,species,5717091433803
4,Narbiakiy,5717091433803,organics,"organic waste, plastics",0.5,0.5,0.5,0.5,BillmanLocal2,species,5717091433803
5,Narbiakiy,5717091433803,organics,"organic waste, plastics",0.5,0.5,0.5,0.5,BillmanLocal2,species,5717091433803
6,Narbiakiy,5717091433803,organics,"organic waste, plastics",0.5,0.5,0.5,0.5,BillmanLocal2,species,5717091433803


In [10]:
locations_df

,name,class,objid,radius,mass,orbitsDistance,orbitsId,orbitsName,isSupportsLife,isPopulated,isHomeworld,username,objtype,id
0,Tadt,terrestrial,3053619110511,0.691,0.329,0.67,4767715660845,Teinrai,true,true,true,BillmanLocal2,planet,3053619110511
1,Tadt,terrestrial,3053619110511,0.691,0.329,0.67,4767715660845,Teinrai,true,true,true,BillmanLocal2,planet,3053619110511
2,Tadt,terrestrial,3053619110511,0.691,0.329,0.67,4767715660845,Teinrai,true,true,true,BillmanLocal2,planet,3053619110511
3,Tadt,terrestrial,3053619110511,0.691,0.329,0.67,4767715660845,Teinrai,true,true,true,BillmanLocal2,planet,3053619110511
4,Tadt,terrestrial,3053619110511,0.691,0.329,0.67,4767715660845,Teinrai,true,true,true,BillmanLocal2,planet,3053619110511
5,Tadt,terrestrial,3053619110511,0.691,0.329,0.67,4767715660845,Teinrai,true,true,true,BillmanLocal2,planet,3053619110511
6,Tadt,terrestrial,3053619110511,0.691,0.329,0.67,4767715660845,Teinrai,true,true,true,BillmanLocal2,planet,3053619110511


In [11]:
factions_df

,name,objid,lat,long,username,objtype,id
0,Tadtperei,3944934639394,0.188,0.0,BillmanLocal2,faction,3944934639394
1,Harpo,3078395587160,-0.188,0.0,BillmanLocal2,faction,3078395587160
2,Harpo,3078395587160,-0.188,0.0,BillmanLocal2,faction,3078395587160
3,Harpo,3078395587160,-0.188,0.0,BillmanLocal2,faction,3078395587160
4,Harpo,3078395587160,-0.188,0.0,BillmanLocal2,faction,3078395587160
5,Harpo,3078395587160,-0.188,0.0,BillmanLocal2,faction,3078395587160
6,Tadtperei,3944934639394,0.188,0.0,BillmanLocal2,faction,3944934639394


In [12]:
pops_df['roll'] = pops_df['objid'].apply(lambda x: np.random.random())
pops_df['grow'] = pops_df[['wealth','health']].T.mean() >= pops_df['roll']

reproducing_pops = pops_df[pops_df['grow']].drop(['roll','grow'],axis=1)
reproducing_pops

,health,name,objid,conformity,literacy,aggression,constitution,isIn,industry,wealth,factionLoyalty,isIdle,username,objtype,id
0,0.7,Tadtperei Wal,9182913651185,0.386,0.484,0.533,0.194,3944934639394,0.3635,0.4238,0.382,true,BillmanLocal2,pop,9182913651185
1,0.7,Harpo Folamso,6961571307402,0.578,0.393,0.345,0.738,3078395587160,0.5415,0.4672,0.490,true,BillmanLocal2,pop,6961571307402
4,0.7,Harpo Manburselto,3770768094710,0.364,0.551,0.548,0.529,3078395587160,0.5385,0.5448,0.081,true,BillmanLocal2,pop,3770768094710
5,0.7,Harpo Pingfrankayyo,4343184883492,0.456,0.656,0.437,0.498,3078395587160,0.4675,0.5618,0.577,true,BillmanLocal2,pop,4343184883492
6,0.7,Tadtperei Melgretanhor,7006358225710,0.485,0.484,0.666,0.326,3944934639394,0.4960,0.4900,0.295,true,BillmanLocal2,pop,7006358225710


In [13]:
print(f"{len(reproducing_pops)} of {len(pops_df)} will grow")

5 of 7 will grow


Going to create species objects for the populations that will reproduce

In [14]:
species = [population.species.Species(species_df.drop_duplicates().T.to_dict()[i]) for i in  species_df.drop_duplicates().T.to_dict().keys()]
species

[<species: None; 5717091433803; Narbiakiy>]

In [20]:
for i in species:
    global_health_manager.species_dict[i.uuid]=i
global_health_manager.species_dict

{'9864646606451': <species: None; 5717091433803; Narbiakiy>,
 '5717091433803': <species: None; 5717091433803; Narbiakiy>}

In [21]:
factions = [population.Faction(factions_df.drop_duplicates().T.to_dict()[i]) for i in  factions_df.drop_duplicates().T.to_dict().keys()]
factions

[<faction: None; 0993925214156; Tadtperei>,
 <faction: None; 5314050308498; Harpo>]

In [22]:
def get_faction(objid):
    res = [i for i in factions if i.objid == objid]
    return res[0]

get_faction(factions[0].objid)

<faction: None; 0993925214156; Tadtperei>

In [23]:
global_health_manager.species_dict

{'9864646606451': <species: None; 5717091433803; Narbiakiy>,
 '5717091433803': <species: None; 5717091433803; Narbiakiy>}

In [24]:
children = []
events = []
event_edges = []
for i in reproducing_pops.index.to_list():
    s = global_health_manager.species_dict[species_df.loc[i].to_dict()['objid']]
    p = reproducing_pops.loc[i].to_dict()
    #adding the current pop as a defautl to the species.
    s.config['defaults'] = p
    f = factions_df.loc[i].to_dict()
    l = locations_df.loc[i].to_dict()
    child = population.Pop(s)
    child.birthplace = l
    child.inhabitsEdge = {"node1": child.objid, "node2": l['objid'], "label": "inhabits"}
    child.name = p['name']+language.make_word(1).lower()
    get_faction(f['objid']).assign_pop_to_faction(child)
    event = global_health_manager.population_growth_event(p,l,child)
    events.append(event)
    event_edges.append({"node1": event['objid'], "node2": l['objid'], "label": "happenedAt"})
    event_edges.append({"node1": p['objid'], "node2": event['objid'], "label": "caused"})
    children.append(child)    

    


In [25]:
pd.DataFrame(events).head()

,objid,name,label,text,visibleTo,time,username
0,7054541174614,population growth,event,The population (Tadtperei Wal) inhabiting Tadt...,BillmanLocal2,1001,event
1,7430774511724,population growth,event,The population (Harpo Folamso) inhabiting Tadt...,BillmanLocal2,1001,event
2,7892996899427,population growth,event,The population (Harpo Manburselto) inhabiting ...,BillmanLocal2,1001,event
3,3385443088526,population growth,event,The population (Harpo Pingfrankayyo) inhabitin...,BillmanLocal2,1001,event
4,8695760882129,population growth,event,The population (Tadtperei Melgretanhor) inhabi...,BillmanLocal2,1001,event


In [26]:
children_df = pd.DataFrame([c.get_data() for c in children])
children_df.head()

,name,objid,label,conformity,literacy,aggression,constitution,health,isIn,industry,wealth,factionLoyalty,isIdle
0,Tadtperei Walder,3961341297890,pop,0.492,0.363,0.494,0.399,0.7,3944934639394,0.4465,0.40475,0.421,True
1,Harpo Folamsomos,5445171423741,pop,0.606,0.549,0.651,0.433,0.7,3078395587160,0.5420,0.54550,0.638,True
2,Harpo Manburseltopol,6887046409113,pop,0.464,0.537,0.275,0.504,0.7,3078395587160,0.3895,0.46325,0.691,True
3,Harpo Pingfrankayyofort,2221035378717,pop,0.355,0.416,0.477,0.611,0.7,3078395587160,0.5440,0.48000,0.495,True
4,Tadtperei Melgretanhornorth,3627355219122,pop,0.473,0.493,0.426,0.606,0.7,3944934639394,0.5160,0.50450,0.577,True


In [27]:
p

{'health': 0.7,
 'name': 'Tadtperei Melgretanhor',
 'objid': '7006358225710',
 'conformity': 0.485,
 'literacy': 0.484,
 'aggression': 0.666,
 'constitution': 0.326,
 'isIn': '3944934639394',
 'industry': 0.496,
 'wealth': 0.49,
 'factionLoyalty': 0.295,
 'isIdle': 'true',
 'username': 'BillmanLocal2',
 'objtype': 'pop',
 'id': '7006358225710'}

In [28]:
child.get_data()

{'name': 'Tadtperei Melgretanhornorth',
 'objid': '3627355219122',
 'label': 'pop',
 'conformity': 0.473,
 'literacy': 0.493,
 'aggression': 0.426,
 'constitution': 0.606,
 'health': 0.7,
 'isIn': '3944934639394',
 'industry': 0.516,
 'wealth': 0.5045,
 'factionLoyalty': 0.577,
 'isIdle': True}

In [29]:
factionedges = []
[f.get_pop_edges(factionedges) for f in factions]
pd.DataFrame(factionedges).head()

,node1,node2,label
0,3961341297890,3944934639394,isIn
1,3627355219122,3944934639394,isIn
2,5445171423741,3078395587160,isIn
3,6887046409113,3078395587160,isIn
4,2221035378717,3078395587160,isIn


In [30]:
childofEdges = [c.childOf for c in children]
pd.DataFrame(childofEdges).head()

,node1,node2,label
0,3961341297890,9182913651185,childOf
1,5445171423741,6961571307402,childOf
2,6887046409113,3770768094710,childOf
3,2221035378717,4343184883492,childOf
4,3627355219122,7006358225710,childOf


In [31]:
isOfEdges = [c.isOfSpecies for c in children]
pd.DataFrame(isOfEdges).head()

,node1,node2,label
0,3961341297890,9864646606451,isOf
1,5445171423741,9864646606451,isOf
2,6887046409113,9864646606451,isOf
3,2221035378717,9864646606451,isOf
4,3627355219122,9864646606451,isOf


In [32]:
inhabitsEdges = [c.inhabitsEdge for c in children]
pd.DataFrame(inhabitsEdges).head()

,node1,node2,label
0,3961341297890,3053619110511,inhabits
1,5445171423741,3053619110511,inhabits
2,6887046409113,3053619110511,inhabits
3,2221035378717,3053619110511,inhabits
4,3627355219122,3053619110511,inhabits


## Validating The Counts of Edges and Nodes Created

In [33]:
assert(len(reproducing_pops) == len(children))
print(f"Expecting {len(reproducing_pops)} children, got {len(children)}")

assert(len(reproducing_pops) == len(isOfEdges))
print(f"Expecting isOfEdges {len(isOfEdges)}")

assert(len(reproducing_pops) == len(inhabitsEdges))
print(f"Expecting inhabitsEdges {len(inhabitsEdges)}")

assert(len(reproducing_pops) == len(childofEdges))
print(f"Expecting childofEdges {len(childofEdges)}")

assert(len(reproducing_pops) == len(factionedges))
print(f"Expecting factionedges {len(factionedges)}")

assert(len(reproducing_pops) == len(events))
print(f"Expecting events {len(events)}")


assert(len(event_edges) == len(events)*2)
print(f"Expecting event_edges {len(event_edges)} (both caused and happenedAt)")


Expecting 5 children, got 5
Expecting isOfEdges 5
Expecting inhabitsEdges 5
Expecting childofEdges 5
Expecting factionedges 5
Expecting events 5
Expecting event_edges 10 (both caused and happenedAt)


## Building the uploaded data

In [34]:
data = {"nodes":[c.get_data() for c in children] + events ,"edges":isOfEdges + inhabitsEdges + childofEdges + factionedges + event_edges }

print(f"The final dataset is {len(data.get('nodes'))} nodes and {len(data.get('edges'))} edges")

c.upload_data('notebook',data)


The final dataset is 10 nodes and 30 edges


It takes a while to upload all of that data. 